In [2]:


# https://open-meteo.com/en/docs"


In [ ]:
import time
import json
import requests
import yaml
from pathlib import Path

OPENMETEO_URL = "https://api.open-meteo.com/v1/forecast"

# 从yaml文件里读地点信息
def load_locations(config_path: str = "config/locations.yaml"):
    with open(config_path, "r", encoding="utf-8") as f:
        cfg = yaml.safe_load(f)
    return cfg["locations"]

# 从Open-Meteo拿天气预报
def fetch_forecast(lat: float, lon: float): # 入参：精度和维度
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "wind_speed_10m,wind_gusts_10m,temperature_2m,precipitation,pressure_msl",
        "forecast_days": 2,
        "timeformat": "unixtime",
        "timezone": "UTC",
    }
    resp = requests.get(OPENMETEO_URL, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json()

def main():
    locations = load_locations()
    for loc_id, coords in locations.items():
        lat = coords["latitude"]
        lon = coords["longitude"]
        data = fetch_forecast(lat, lon)
        print(f"=== Sample forecast for {loc_id} ===")
        print(json.dumps({k: data.get(k) for k in ["latitude", "longitude", "timezone"]}, indent=2))
        # TODO: transform `data` into per-hour Kafka messages
        # TODO: send to Kafka topic `weather_iceland_raw`

if __name__ == "__main__":
    main()
